# Chamgab ML Model Retraining

PC 종료 후에도 클라우드에서 모델을 재훈련할 수 있는 노트북입니다.

## 파이프라인
1. 레포 클론 + 의존성 설치
2. Supabase에서 훈련 데이터 fetch
3. Feature Engineering (71개 피처)
4. XGBoost 학습 (Early Stopping + TimeSeriesSplit CV)
5. SHAP Explainer 생성
6. 모델 저장 (.pkl)
7. HuggingFace Spaces 배포 (선택)
8. Supabase 분석 재생성 (선택)

## 소요 시간
- 데이터 fetch: ~5분
- 학습: ~10-20분
- 분석 재생성: ~30분
- **총: ~1시간 이내**

---
## 0. 환경 설정

아래 셀에 Supabase 키를 입력하세요. Colab Secrets 사용을 권장합니다.

In [23]:
# ============================================================
# 방법 1: Colab Secrets (권장)
# 왼쪽 사이드바 > 열쇠 아이콘 > 시크릿 추가
# ============================================================
import os

try:
    from google.colab import userdata
    os.environ['SUPABASE_URL'] = userdata.get('SUPABASE_URL')
    os.environ['SUPABASE_SERVICE_KEY'] = userdata.get('SUPABASE_SERVICE_KEY')
    # 선택: HuggingFace 배포용
    try:
        os.environ['HF_TOKEN'] = userdata.get('HF_TOKEN')
    except Exception:
        pass
    print('Colab Secrets에서 키 로드 완료')
except Exception:
    pass

# ============================================================
# 방법 2: 직접 입력 (Secrets 미사용 시 주석 해제)
# ============================================================
# os.environ['SUPABASE_URL'] = 'https://YOUR_PROJECT.supabase.co'
# os.environ['SUPABASE_SERVICE_KEY'] = 'eyJ...'  # service_role key

# 확인
assert os.environ.get('SUPABASE_URL'), 'SUPABASE_URL이 설정되지 않았습니다!'
assert os.environ.get('SUPABASE_SERVICE_KEY'), 'SUPABASE_SERVICE_KEY가 설정되지 않았습니다!'
print(f'Supabase URL: {os.environ["SUPABASE_URL"][:40]}...')

Colab Secrets에서 키 로드 완료
Supabase URL: https://csnmpkixzzszuxcsdwou.supabase.co...


---
## 1. 레포 클론 + 의존성 설치

In [24]:
%%bash
# git-lfs 설치 (모델 .pkl 파일 다운로드에 필요)
apt-get install -qq git-lfs
git lfs install

# 기존 chamgab 디렉토리 삭제 후 새로 클론
rm -rf /content/chamgab
GIT_LFS_SKIP_SMUDGE=1 git clone https://github.com/choiwjun/chamgab.git /content/chamgab

# 최신 브랜치로 전환
cd /content/chamgab && git checkout feature/phase6-advancement && git pull origin feature/phase6-advancement

# LFS 파일 다운로드 시도 (실패해도 계속 진행)
cd /content/chamgab && git lfs pull || echo "LFS pull failed, continuing without .pkl files"

Updated git hooks.
Git LFS initialized.
LFS pull failed, continuing without .pkl files


Cloning into '/content/chamgab'...
fatal: Unable to read current working directory: No such file or directory
bash: line 10: cd: /content/chamgab: No such file or directory
bash: line 13: cd: /content/chamgab: No such file or directory


In [25]:
# ML API 의존성 설치
!pip install -q xgboost>=2.0.0 shap>=0.45.0 scikit-learn>=1.4.0 \
    pandas>=2.2.0 numpy>=1.26.0 supabase>=2.0.0 python-dotenv>=1.0.0 \
    httpx>=0.27.0 optuna>=3.5.0 lightgbm>=4.0.0 pyarrow>=15.0.0 \
    lxml>=5.0.0 psycopg2-binary>=2.9.0

print('\n의존성 설치 완료!')

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
/bin/bash: line 1: =2.0.0: No such file or directory

의존성 설치 완료!


In [26]:
# 작업 디렉토리 설정
import os
os.chdir('/content/chamgab/ml-api')

# .env 파일 생성 (Colab 환경변수 → .env)
with open('.env', 'w') as f:
    f.write(f'SUPABASE_URL={os.environ["SUPABASE_URL"]}\n')
    f.write(f'SUPABASE_SERVICE_KEY={os.environ["SUPABASE_SERVICE_KEY"]}\n')

print(f'작업 디렉토리: {os.getcwd()}')
print(f'.env 파일 생성 완료')

FileNotFoundError: [Errno 2] No such file or directory: '/content/chamgab/ml-api'

---
## 2. 모델 재훈련

3가지 모드 중 선택:
- **기본**: Early Stopping + CV (~10분)
- **튜닝**: Optuna 200 trials (~30분)
- **앙상블**: XGBoost + LightGBM (~15분)

In [ ]:
# ============================================================
# 훈련 모드 선택
# ============================================================
BASIC_TRAIN = True       # 기본 학습 (~10분)
OPTUNA_TUNE = True       # 하이퍼파라미터 튜닝 (~30분)
ENSEMBLE = True          # LightGBM 앙상블 (~15분)
TUNE_TRIALS = 200        # Optuna trials 수

# 훈련 커맨드 구성
cmd = 'python -m scripts.train_model'
if OPTUNA_TUNE:
    cmd += f' --tune --trials {TUNE_TRIALS}'
if ENSEMBLE:
    cmd += ' --ensemble'

print(f'실행 커맨드: {cmd}')
print(f'예상 소요: {"~45분" if OPTUNA_TUNE and ENSEMBLE else "~30분" if OPTUNA_TUNE else "~15분" if ENSEMBLE else "~10분"}')

In [ ]:
# 모델 학습 실행
import subprocess
import sys

result = subprocess.run(
    cmd.split(),
    cwd='/content/chamgab/ml-api',
    capture_output=False,
    text=True,
    env={**os.environ, 'PYTHONPATH': '/content/chamgab/ml-api'}
)

if result.returncode == 0:
    print('\n' + '='*60)
    print('모델 학습 성공!')
    print('='*60)
else:
    print(f'\n학습 실패 (exit code: {result.returncode})')

---
## 3. 학습 결과 확인

In [ ]:
import json

metrics_path = '/content/chamgab/ml-api/app/models/apartment_model_metrics.json'

with open(metrics_path, 'r', encoding='utf-8') as f:
    metrics = json.load(f)

print('=== 학습 결과 ===')
print(f'학습 시간: {metrics.get("training_duration_seconds", "N/A")}초')
ds = metrics.get('data_summary', {})
print(f'Train: {ds.get("train_samples", "N/A")}건')
print(f'Val:   {ds.get("val_samples", "N/A")}건')
print(f'Test:  {ds.get("test_samples", "N/A")}건')
print(f'Features: {ds.get("features", "N/A")}개')
print()
print('=== 평가 지표 ===')
m = metrics.get('metrics', {})
print(f'MAE:  {m.get("mae", 0):,.0f} 원')
print(f'RMSE: {m.get("rmse", 0):,.0f} 원')
print(f'R2:   {m.get("r2", 0):.4f}')
print(f'MAPE: {m.get("mape", 0):.2f}%')
print()
if 'cross_validation' in metrics:
    cv = metrics['cross_validation']
    print('=== 교차 검증 ===')
    print(f'CV MAPE: {cv.get("cv_mape_mean", 0):.2f}% +/- {cv.get("cv_mape_std", 0):.2f}%')
    print()
if 'feature_importance_top20' in metrics:
    print('=== Feature Importance Top 10 ===')
    for i, fi in enumerate(metrics['feature_importance_top20'][:10], 1):
        bar = '#' * int(fi['importance'] * 200)
        print(f'{i:2d}. {fi["feature"]:30s} {fi["importance"]:.4f} {bar}')
else:
    print('(Feature importance 정보 없음)')

In [ ]:
# 모델 파일 크기 확인
import os

models_dir = '/content/chamgab/ml-api/app/models'
pkl_files = ['xgboost_model.pkl', 'shap_explainer.pkl', 'feature_artifacts.pkl', 'residual_info.pkl']

print('=== 모델 파일 ===')
total = 0
for f in pkl_files:
    path = os.path.join(models_dir, f)
    if os.path.exists(path):
        size = os.path.getsize(path)
        total += size
        print(f'  {f:30s} {size/1024:,.1f} KB')
    else:
        print(f'  {f:30s} (없음)')
print(f'  {"합계":30s} {total/1024/1024:,.1f} MB')

---
## 4. 모델 다운로드 / HuggingFace 배포

학습된 모델을 가져가는 3가지 방법:

In [ ]:
# 방법 A: Google Drive에 저장 (가장 간단)
SAVE_TO_DRIVE = True  # False로 변경하면 스킵

if SAVE_TO_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')

    import shutil
    dest = '/content/drive/MyDrive/chamgab-models'
    os.makedirs(dest, exist_ok=True)

    for f in pkl_files + ['apartment_model_metrics.json']:
        src = os.path.join(models_dir, f)
        if os.path.exists(src):
            shutil.copy2(src, dest)
            print(f'  복사: {f} -> {dest}/')

    print(f'\nGoogle Drive 저장 완료: {dest}')

In [ ]:
# ============================================================
# [FIX 1] train_model.py 버그 수정: except에서 sys.exit(1) 추가
# ============================================================
import os
os.chdir('/content/chamgab/ml-api')

train_model_path = 'scripts/train_model.py'
with open(train_model_path, 'r', encoding='utf-8') as f:
    content = f.read()

    # Fix: except에서 sys.exit(1) 추가하여 실패 시 비정상 종료
    old_except = '''    except Exception as e:
            print(f"Error: {e}")
                    import traceback
                            traceback.print_exc()'''

                            new_except = '''    except Exception as e:
                                    print(f"Error: {e}")
                                            import traceback
                                                    traceback.print_exc()
                                                            sys.exit(1)  # ★ Fix: 학습 실패 시 비정상 종료'''

                                                            if old_except in content:
                                                                content = content.replace(old_except, new_except)
                                                                    with open(train_model_path, 'w', encoding='utf-8') as f:
                                                                            f.write(content)
                                                                                print('[OK] train_model.py 수정 완료: except에 sys.exit(1) 추가')
                                                                                else:
                                                                                    print('[INFO] 이미 수정되었거나 패턴 불일치')

                                                                                    # ============================================================
                                                                                    # [FIX 2] batch_generate_analyses.py: 모델 존재 확인을 삭제보다 먼저
                                                                                    # ============================================================
                                                                                    batch_path = 'scripts/batch_generate_analyses.py'
                                                                                    with open(batch_path, 'r', encoding='utf-8') as f:
                                                                                        batch_content = f.read()

                                                                                        # 기존: delete → area_map → sync → model load
                                                                                        # 수정: model 존재 확인 → delete → area_map → sync → model load
                                                                                        old_main_flow = '''    # Step 0: 기존 분석 삭제 (--regenerate)
                                                                                            if args.regenerate:
                                                                                                    delete_existing_analyses(sb)

                                                                                                        # Step 0.5: 단지별 대표 면적 pre-fetch
                                                                                                            area_map = fetch_area_by_complex(sb)'''

                                                                                                            new_main_flow = '''    # ★ Fix: 모델 파일 존재 확인을 삭제보다 먼저 수행 (데이터 유실 방지)
                                                                                                                print("\\n[사전 확인] 모델 파일 존재 여부 체크")
                                                                                                                    if not os.path.exists(XGB_MODEL_PATH):
                                                                                                                            print(f"FATAL: 모델 파일 없음: {XGB_MODEL_PATH}")
                                                                                                                                    print("데이터 삭제를 수행하지 않습니다. 먼저 모델을 학습하세요.")
                                                                                                                                            sys.exit(1)
                                                                                                                                                if not os.path.exists(FEATURE_ARTIFACTS_PATH):
                                                                                                                                                        print(f"FATAL: Feature artifacts 없음: {FEATURE_ARTIFACTS_PATH}")
                                                                                                                                                                print("데이터 삭제를 수행하지 않습니다. 먼저 모델을 학습하세요.")
                                                                                                                                                                        sys.exit(1)
                                                                                                                                                                            print("  모델 파일 확인 OK")

                                                                                                                                                                                # Step 0: 기존 분석 삭제 (--regenerate)
                                                                                                                                                                                    if args.regenerate:
                                                                                                                                                                                            delete_existing_analyses(sb)

                                                                                                                                                                                                # Step 0.5: 단지별 대표 면적 pre-fetch
                                                                                                                                                                                                    area_map = fetch_area_by_complex(sb)'''

                                                                                                                                                                                                    if old_main_flow in batch_content:
                                                                                                                                                                                                        batch_content = batch_content.replace(old_main_flow, new_main_flow)
                                                                                                                                                                                                            with open(batch_path, 'w', encoding='utf-8') as f:
                                                                                                                                                                                                                    f.write(batch_content)
                                                                                                                                                                                                                        print('[OK] batch_generate_analyses.py 수정 완료: 모델 확인을 삭제 전으로 이동')
                                                                                                                                                                                                                        else:
                                                                                                                                                                                                                            print('[INFO] 이미 수정되었거나 패턴 불일치')

                                                                                                                                                                                                                            print('\n=== 스크립트 수정 완료 ===')

In [ ]:
# 방법 B: HuggingFace Spaces에 직접 배포
DEPLOY_TO_HF = False  # True로 변경하면 실행
HF_REPO = 'YOUR_USERNAME/chamgab-ml-api'  # HuggingFace Spaces repo

if DEPLOY_TO_HF:
    !pip install -q huggingface_hub

    from huggingface_hub import HfApi

    hf_token = os.environ.get('HF_TOKEN')
    assert hf_token, 'HF_TOKEN이 설정되지 않았습니다!'

    api = HfApi(token=hf_token)

    for f in pkl_files:
        local_path = os.path.join(models_dir, f)
        if os.path.exists(local_path):
            api.upload_file(
                path_or_fileobj=local_path,
                path_in_repo=f'app/models/{f}',
                repo_id=HF_REPO,
                repo_type='space',
            )
            print(f'  업로드: {f}')

    print(f'\nHuggingFace Spaces 배포 완료: {HF_REPO}')
    print('Space가 자동으로 재빌드됩니다.')

In [ ]:
%%bash
cd /content/chamgab/ml-api

# FIX 1: train_model.py - except에서 sys.exit(1) 추가
sed -i '/        traceback.print_exc()$/a\        sys.exit(1)  # Fix: exit on error' scripts/train_model.py
echo "[FIX 1] train_model.py: sys.exit(1) 추가 완료"

# FIX 2: batch_generate_analyses.py - 모델 존재 확인을 삭제 전으로 이동
# 삭제 전에 모델 파일 확인하는 코드 삽입
sed -i '/    # Step 0: 기존 분석 삭제 (--regenerate)/i\    # Fix: 모델 파일 존재 확인 (삭제 전)\n    print("\\n[사전 확인] 모델 파일 존재 여부 체크")\n    if not os.path.exists(XGB_MODEL_PATH):\n        print(f"FATAL: 모델 파일 없음: {XGB_MODEL_PATH}")\n        print("데이터 삭제를 수행하지 않습니다. 먼저 모델을 학습하세요.")\n        sys.exit(1)\n    if not os.path.exists(FEATURE_ARTIFACTS_PATH):\n        print(f"FATAL: Feature artifacts 없음: {FEATURE_ARTIFACTS_PATH}")\n        print("데이터 삭제를 수행하지 않습니다. 먼저 모델을 학습하세요.")\n        sys.exit(1)\n    print("  모델 파일 확인 OK")\n' scripts/batch_generate_analyses.py
echo "[FIX 2] batch_generate_analyses.py: 모델 존재 확인 코드 추가 완료"

# 수정 결과 확인
echo ""
echo "=== train_model.py except 부분 확인 ==="
grep -n -A2 "traceback.print_exc" scripts/train_model.py | head -10

echo ""
echo "=== batch_generate_analyses.py 사전 확인 부분 ==="
grep -n -A3 "사전 확인" scripts/batch_generate_analyses.py | head -15

In [ ]:
# 방법 C: 직접 다운로드 (브라우저)
DOWNLOAD_FILES = False  # True로 변경하면 실행

if DOWNLOAD_FILES:
    from google.colab import files

    # ZIP으로 묶어서 다운로드
    import zipfile
    zip_path = '/content/chamgab_models.zip'
    with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zf:
        for f in pkl_files + ['apartment_model_metrics.json']:
            src = os.path.join(models_dir, f)
            if os.path.exists(src):
                zf.write(src, f)

    files.download(zip_path)
    print('다운로드 시작!')

In [ ]:
# ============================================================
# [Step 1] 모델 재학습 (수정된 스크립트로)
# ============================================================
import subprocess, os

os.chdir('/content/chamgab/ml-api')

# 기본 학습만 먼저 (빠르게 ~10분) - 모델 파일 생성 확인용
cmd = 'python -m scripts.train_model'
print(f'실행: {cmd}')
print('=' * 60)

result = subprocess.run(
    cmd.split(),
        cwd='/content/chamgab/ml-api',
            capture_output=False,
                text=True,
                    env={**os.environ, 'PYTHONPATH': '/content/chamgab/ml-api'}
                    )

                    print('=' * 60)
                    if result.returncode == 0:
                        print('모델 학습 성공!')
                            # 모델 파일 존재 확인
                                models_dir = '/content/chamgab/ml-api/app/models'
                                    for f in ['xgboost_model.pkl', 'shap_explainer.pkl', 'feature_artifacts.pkl', 'residual_info.pkl']:
                                            path = os.path.join(models_dir, f)
                                                    if os.path.exists(path):
                                                                size = os.path.getsize(path) / 1024
                                                                            print(f'  [OK] {f}: {size:.1f} KB')
                                                                                    else:
                                                                                                print(f'  [FAIL] {f}: 없음!')
                                                                                                else:
                                                                                                    print(f'학습 실패 (exit code: {result.returncode})')
                                                                                                        print('에러를 확인하고 다시 시도하세요.')

---
## 5. (선택) Supabase 분석 데이터 재생성

새 모델로 chamgab_analyses + price_factors를 재생성합니다.

**주의**: 전체 25,890건 재생성에 약 30분 소요됩니다.

In [ ]:
%%bash
echo "[Step 1] 모델 재학습 시작 (기본 모드, ~10분)"
cd /content/chamgab/ml-api
PYTHONPATH=/content/chamgab/ml-api python -m scripts.train_model
echo ""
echo "[Step 2] 모델 파일 확인"
ls -la app/models/*.pkl app/models/*.json 2>/dev/null || echo "모델 파일 없음!"

In [ ]:
# 분석 재생성 실행 여부
REGENERATE_ANALYSES = True  # 새 모델로 전체 재생성

if REGENERATE_ANALYSES:
    print('chamgab_analyses + price_factors 재생성 시작...')
    print('기존 데이터를 모두 삭제하고 새 모델로 재생성합니다.')
    print('예상 소요: ~30분')
    print('='*60)

    !cd /content/chamgab/ml-api && python -m scripts.batch_generate_analyses --regenerate --skip-sync

    print('\n재생성 완료!')
else:
    print('재생성 스킵 (REGENERATE_ANALYSES = False)')
    print('나중에 로컬에서 실행: python -m scripts.batch_generate_analyses --regenerate --skip-sync')

---
## 6. (선택) 재생성 결과 검증

In [ ]:
# 재생성 후 검증
if REGENERATE_ANALYSES:
    from supabase import create_client

    sb = create_client(os.environ['SUPABASE_URL'], os.environ['SUPABASE_SERVICE_KEY'])

    # 분석 건수
    analyses = sb.table('chamgab_analyses').select('id', count='exact').execute()
    factors = sb.table('price_factors').select('id', count='exact').execute()

    print(f'chamgab_analyses: {analyses.count}건')
    print(f'price_factors: {factors.count}건')

    # confidence 분포
    sample = sb.table('chamgab_analyses').select('confidence, chamgab_price').limit(1000).execute()
    import pandas as pd
    df = pd.DataFrame(sample.data)
    print(f'\nconfidence 분포:')
    print(f'  min: {df["confidence"].min()}')
    print(f'  median: {df["confidence"].median()}')
    print(f'  max: {df["confidence"].max()}')

    # 가격 비교 (최근 거래가 vs AI예측가)
    sample_with_price = sb.table('chamgab_analyses').select(
        'chamgab_price, property_id'
    ).limit(100).execute()

    print('\n검증 완료!')

---
## 사용 가이드

### Colab Secrets 설정 방법
1. 왼쪽 사이드바에서 열쇠 아이콘 클릭
2. `SUPABASE_URL` 추가 (값: `https://xxx.supabase.co`)
3. `SUPABASE_SERVICE_KEY` 추가 (값: `eyJ...`)
4. (선택) `HF_TOKEN` 추가 (HuggingFace 배포용)

### 월간 재훈련 루틴
1. 이 노트북을 Google Drive에 저장
2. 매월 1일 접속해서 **Run All** (Ctrl+F9)
3. 결과 확인 후 HuggingFace 배포

### PC 종료 시 유의사항
- 브라우저 닫아도 **최대 12시간** 실행 유지 (무료)
- Colab Pro: 최대 24시간
- 세션 만료 시 Google Drive에 저장된 모델은 유지됨

In [ ]:
%%writefile /content/chamgab/ml-api/scripts/patch_fe.py
"""feature_engineering.py의 _load_from_database를 JOIN 없이 분리 fetch로 패치"""
import re

FE_PATH = '/content/chamgab/ml-api/scripts/feature_engineering.py'

# 새로운 _load_from_database 메서드 (JOIN 없이 분리 fetch + pandas merge)
NEW_METHOD = '''    def _load_from_database(self) -> pd.DataFrame:
        """Supabase에서 학습 데이터 로드 (분리 fetch - 타임아웃 방지)"""
                client = get_supabase_client()

                        # 1) transactions만 가져오기 (JOIN 없이, 작은 page_size)
                                all_txns = []
                                        page_size = 500
                                                offset = 0
                                                        while True:
                                                                    try:
                                                                                    result = client.table("transactions").select(
                                                                                                        "id, transaction_date, price, area_exclusive, floor, dong, "
                                                                                                                            "region_code, apt_name, sigungu, property_id, complex_id"
                                                                                                                                            ).range(offset, offset + page_size - 1).execute()
                                                                                                                                                        except Exception as e:
                                                                                                                                                                        print(f"  transactions fetch 오류 (offset={offset}): {e}")
                                                                                                                                                                                        import time; time.sleep(2)
                                                                                                                                                                                                        try:
                                                                                                                                                                                                                            result = client.table("transactions").select(
                                                                                                                                                                                                                                                    "id, transaction_date, price, area_exclusive, floor, dong, "
                                                                                                                                                                                                                                                                            "region_code, apt_name, sigungu, property_id, complex_id"
                                                                                                                                                                                                                                                                                                ).range(offset, offset + page_size - 1).execute()
                                                                                                                                                                                                                                                                                                                except Exception:
                                                                                                                                                                                                                                                                                                                                    break
                                                                                                                                                                                                                                                                                                                                                if not result.data:
                                                                                                                                                                                                                                                                                                                                                                break
                                                                                                                                                                                                                                                                                                                                                                            all_txns.extend(result.data)
                                                                                                                                                                                                                                                                                                                                                                                        if len(result.data) < page_size:
                                                                                                                                                                                                                                                                                                                                                                                                        break
                                                                                                                                                                                                                                                                                                                                                                                                                    offset += page_size
                                                                                                                                                                                                                                                                                                                                                                                                                                if offset % 10000 == 0:
                                                                                                                                                                                                                                                                                                                                                                                                                                                print(f"  transactions 로드: {offset}건...")

                                                                                                                                                                                                                                                                                                                                                                                                                                                        if not all_txns:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                    print("거래 데이터가 없습니다.")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                return pd.DataFrame()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        print(f"transactions: {len(all_txns)}건 로드 완료")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # 2) properties 가져오기
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        all_props = []
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                offset = 0
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        while True:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    try:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    result = client.table("properties").select(
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        "id, name, address, sido, sigungu, eupmyeondong, "
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            "area_exclusive, built_year, floors, property_type, complex_id"
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            ).range(offset, offset + page_size - 1).execute()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        except Exception:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        break
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    if not result.data:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    break
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                all_props.extend(result.data)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            if len(result.data) < page_size:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            break
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        offset += page_size
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                print(f"properties: {len(all_props)}건 로드 완료")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        # 3) complexes 가져오기
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                all_complexes = []
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        offset = 0
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                while True:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            try:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            result = client.table("complexes").select(
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                "id, name, total_units, total_buildings, built_year, parking_ratio, brand"
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                ).range(offset, offset + page_size - 1).execute()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            except Exception:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            break
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if not result.data:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        break
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    all_complexes.extend(result.data)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                if len(result.data) < page_size:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                break
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            offset += page_size
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    print(f"complexes: {len(all_complexes)}건 로드 완료")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # 4) pandas merge
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    txn_df = pd.DataFrame(all_txns)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            prop_df = pd.DataFrame(all_props) if all_props else pd.DataFrame()
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    comp_df = pd.DataFrame(all_complexes) if all_complexes else pd.DataFrame()

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            # properties merge (property_id 기준)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    if not prop_df.empty and "property_id" in txn_df.columns:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                prop_df = prop_df.rename(columns={
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                "id": "prop_id", "sido": "prop_sido", "sigungu": "prop_sigungu",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                "eupmyeondong": "prop_eupmyeondong", "built_year": "prop_built_year",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                "floors": "prop_floors", "property_type": "prop_type",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                "area_exclusive": "prop_area_exclusive",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                "complex_id": "prop_complex_id"
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            })
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        txn_df = txn_df.merge(prop_df, left_on="property_id", right_on="prop_id", how="left")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # complexes merge (complex_id 기준)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        if not comp_df.empty and "complex_id" in txn_df.columns:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    comp_df = comp_df.rename(columns={
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    "id": "comp_id", "name": "complex_name",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    "total_units": "complex_total_units",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    "total_buildings": "complex_total_buildings",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    "built_year": "complex_built_year",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    "parking_ratio": "complex_parking_ratio",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    "brand": "complex_brand"
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                })
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            txn_df = txn_df.merge(comp_df, left_on="complex_id", right_on="comp_id", how="left")

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    print(f"총 {len(txn_df)}건 merge 완료 (Supabase)")
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            return txn_df
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            '''

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            with open(FE_PATH, 'r', encoding='utf-8') as f:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                content = f.read()

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # 기존 _load_from_database 메서드 찾기 및 교체
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                # 메서드 시작부터 다음 메서드(def create_features) 직전까지
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                pattern = r'    def _load_from_database\(self\) -> pd\.DataFrame:.*?(?=    def create_features)'
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                match = re.search(pattern, content, re.DOTALL)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                if match:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    content = content[:match.start()] + NEW_METHOD + '\n' + content[match.end():]
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        with open(FE_PATH, 'w', encoding='utf-8') as f:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                f.write(content)
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    print('[OK] _load_from_database 패치 완료: JOIN 제거 + 분리 fetch + pandas merge')
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    else:
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        print('[FAIL] 패턴 매칭 실패 - 수동 수정 필요')


In [ ]:
%%bash
echo "=== [FIX 3] feature_engineering.py 패치 적용 ==="
cd /content/chamgab/ml-api
python scripts/patch_fe.py

echo ""
echo "=== 패치 확인: _load_from_database 첫 줄 ==="
grep -n "def _load_from_database" scripts/feature_engineering.py
grep -n "분리 fetch" scripts/feature_engineering.py

echo ""
echo "=== [Step 1] 모델 재학습 시작 (기본 모드) ==="
PYTHONPATH=/content/chamgab/ml-api python -m scripts.train_model

echo ""
echo "=== [Step 2] 모델 파일 확인 ==="
ls -la app/models/*.pkl 2>/dev/null && echo "모델 파일 생성 OK!" || echo "FAIL: 모델 파일 없음"